In [1]:

import gym
from gym import logger as gymlogger
from gym.wrappers import RecordVideo
gymlogger.set_level(40) #error only
#import tensorflow as tf
import numpy as np
import random
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline
import math
import glob
import io
import base64
from IPython.display import HTML
from IPython import display as ipythondisplay

def show_video():
  mp4list = glob.glob('video/*.mp4')
  if len(mp4list) > 0:
    mp4 = mp4list[0]
    video = io.open(mp4, 'r+b').read()
    encoded = base64.b64encode(video)
    ipythondisplay.display(HTML(data='''<video alt="test" autoplay 
                loop controls style="height: 400px;">
                <source src="data:video/mp4;base64,{0}" type="video/mp4" />
             </video>'''.format(encoded.decode('ascii'))))
  else: 
    print("Could not find video")

In [ ]:
env = gym.make("CartPole-v1")

## Initialize variables

In [4]:
LEARNING_RATE = 0.1

DISCOUNT = 0.90
EPISODES = 60000
total = 0
total_reward = 0


Observation = [40, 40, 85, 50]
np_array_win_size = np.array([0.05, 0.25, 0.01, 0.1])

#epsilon = 1

obs = env.reset()


new_env = env.step(0)[0]
print(round(new_env[0], 2))

0.03


## Create Q table

In [5]:
q_table = np.random.uniform(low=0, high=1, size=(Observation + [env.action_space.n]))
q_table.shape

(40, 40, 85, 50, 2)

## Getting discrete state

In [6]:
def get_discrete_state(state):
    
    #Given a state, round to the nearest 0.25
    #divide by the window size to find which bucket it belongs to
    discrete_state =np.array([np.round(state[0] *20) /20, np.round(state[1] *4) /4, np.round(state[2], 2) ,np.round(state[3], 2)])
    new = discrete_state/np_array_win_size
    
    return tuple(new.astype(int))


print(get_discrete_state([1,-0.4,0.8,0.4]))

(20, -2, 80, 4)


## Running

In [ ]:
for episode in range(EPISODES): 
   
    discrete_state = get_discrete_state(env.reset()[0])  
    #print(discrete_state)
    done = False
    episode_reward = 0 
    
    #update every 2000 episodes
    if episode % 2000 == 0: 
        print("Episode: " + str(episode))

    while not done: 
        #Always get max for now
        #if np.random.random() > epsilon:
        action = np.argmax(q_table[discrete_state]) 
        #else:
        #action = np.random.randint(0, env.action_space.n) 

        new_state = env.step(action)[0]
        reward = env.step(action)[1]
        done = env.step(action)[2]
        episode_reward += reward 

        new_discrete_state = get_discrete_state(new_state)

        if not done: 
            
            max_future_q = np.max(q_table[new_discrete_state])

            current_q = q_table[discrete_state + (action,)]

            new_q = (1-LEARNING_RATE)*current_q + LEARNING_RATE * (reward + DISCOUNT * max_future_q)
            q_table[discrete_state + (action,)] = new_q

        discrete_state = new_discrete_state
  

    total_reward += episode_reward
    

    if episode % 1000 == 0: 

        mean_reward = total_reward / 1000
        print("Mean Reward: " + str(mean_reward))
        total_reward = 0

env.close()

Episode: 0
Mean Reward: 0.003
Mean Reward: 4.667
Episode: 2000
Mean Reward: 4.869
Mean Reward: 4.843
Episode: 4000
Mean Reward: 4.738
Mean Reward: 4.84
Episode: 6000
Mean Reward: 4.671
Mean Reward: 4.741
Episode: 8000
Mean Reward: 4.647
Mean Reward: 4.726
Episode: 10000
Mean Reward: 4.708
Mean Reward: 4.792
Episode: 12000
Mean Reward: 4.825
Mean Reward: 4.867
Episode: 14000
Mean Reward: 4.838
Mean Reward: 4.847
Episode: 16000
Mean Reward: 4.804
Mean Reward: 4.647
Episode: 18000
Mean Reward: 4.867
Mean Reward: 4.83
Episode: 20000
Mean Reward: 4.707
Mean Reward: 4.859
Episode: 22000
Mean Reward: 4.789
Mean Reward: 4.707
Episode: 24000
Mean Reward: 4.804
Mean Reward: 4.822
Episode: 26000
Mean Reward: 4.741
Mean Reward: 4.772
Episode: 28000
Mean Reward: 4.784
Mean Reward: 4.747
Episode: 30000
Mean Reward: 4.735
Mean Reward: 4.773
Episode: 32000
Mean Reward: 4.695
Mean Reward: 4.828
Episode: 34000
Mean Reward: 4.682
Mean Reward: 4.879
